#### Pacotes importados para manipulação dos dados

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopy.distance
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

##### Importação dos dados

In [3]:
sample_submission = pd.read_csv('dados/taxi-sample-submission.csv')
submission = pd.read_csv('dados/taxi-submission.csv')
taxi_train = pd.read_csv('dados/taxi-train.csv')

#### Tratamento

In [4]:
taxi_train.head()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location,ride_date
0,5c41b7afe9da0d7fbe60dc0e925975b832375a3c,efd204f38a83a59df5bf4a1fe004aa291183e235ee40ee...,2016-02-07 01:45:00+00:00,2016-02-07 01:45:00+00:00,360.0,0.0,1.703108e+10,1.703108e+10,8.0,8.0,...,8.50,Credit Card,Northwest Management LLC,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.900221,-87.629105,POINT (-87.6291051864 41.9002212967),2016-02-07
1,9333962b69bef1506be0b0a7edbd1d6626edd6ab,1a1189d1e9f4f676bbeaf49903912684b94453576a234f...,2013-05-08 19:15:00+00:00,2013-05-08 19:30:00+00:00,840.0,5.4,NaN,NaN,NaN,NaN,...,13.85,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-05-08
2,6329807d0ad342bfe6671b2553592edadfd742b6,25e9bbb9704eb82311a4d4036baba6451cd27138c4d6eb...,2013-02-17 12:30:00+00:00,2013-02-17 12:30:00+00:00,NaN,0.0,NaN,NaN,NaN,NaN,...,52.38,Credit Card,Chicago Elite Cab Corp.,NaN,NaN,NaN,NaN,NaN,NaN,2013-02-17
3,049949c0f58cc75d58ef9faf8ba4030c6ef0cf34,8b8b185d8a60ac5edd94c0494d0b9b9c9c86d54c0ab3dd...,2014-11-09 10:45:00+00:00,2014-11-09 11:00:00+00:00,540.0,1.6,NaN,NaN,6.0,3.0,...,9.55,Credit Card,NaN,41.944227,-87.655998,POINT (-87.6559981815 41.9442266014),41.965812,-87.655879,POINT (-87.6558787862 41.96581197),2014-11-09
4,4554e53540678f18384d5fcd6d42623ccc60d6e2,73b2f5adecea91eeef3900303a07f1b0519a594cffb6b0...,2019-04-13 20:00:00+00:00,2019-04-13 20:00:00+00:00,577.0,1.5,1.703184e+10,1.703108e+10,32.0,8.0,...,11.25,Credit Card,Chicago Taxicab,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-04-13


In [5]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   unique_key              20000 non-null  object 
 1   taxi_id                 20000 non-null  object 
 2   trip_start_timestamp    20000 non-null  object 
 3   trip_end_timestamp      19995 non-null  object 
 4   trip_seconds            19870 non-null  float64
 5   trip_miles              20000 non-null  float64
 6   pickup_census_tract     13124 non-null  float64
 7   dropoff_census_tract    13012 non-null  float64
 8   pickup_community_area   17672 non-null  float64
 9   dropoff_community_area  17282 non-null  float64
 10  tips                    19999 non-null  float64
 11  tolls                   16639 non-null  float64
 12  extras                  19999 non-null  float64
 13  trip_total              19999 non-null  float64
 14  payment_type            20000 non-null

In [6]:
taxi_train.describe()

,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
count,29805.000000,30000.000000,1.966300e+04,1.955900e+04,26445.000000,25919.000000,29999.000000,29999.000000,24933.000000,29999.000000,29999.000000,26449.000000,26449.000000,26007.000000,26007.000000
mean,798.112833,3.237427,1.703138e+10,1.703136e+10,23.859217,21.947375,12.872966,1.402873,0.001043,0.867869,15.191000,41.901371,-87.659851,41.901145,-87.655315
std,1507.877805,6.297811,3.431865e+05,3.375077e+05,20.037496,18.160866,12.289114,2.640751,0.055194,2.742169,15.147028,0.039336,0.073250,0.039685,0.061020
min,0.000000,0.000000,1.703101e+10,1.703101e+10,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,41.673820,-87.913625,41.660136,-87.913625
25%,310.000000,0.200000,1.703108e+10,1.703108e+10,8.000000,8.000000,6.000000,0.000000,0.000000,0.000000,7.000000,41.880994,-87.655998,41.880994,-87.655998
50%,540.000000,1.190000,1.703128e+10,1.703128e+10,24.000000,22.000000,8.050000,0.000000,0.000000,0.000000,9.650000,41.892073,-87.632746,41.892508,-87.633308
75%,960.000000,3.000000,1.703184e+10,1.703183e+10,32.000000,32.000000,13.500000,2.000000,0.000000,1.000000,15.450000,41.914616,-87.626211,41.921701,-87.626211
max,86386.000000,247.000000,1.703198e+10,1.703198e+10,77.000000,77.000000,269.850000,63.000000,4.000000,99.500000,274.350000,42.009623,-87.534903,42.015934,-87.540936


In [7]:
submission.isnull().sum()[taxi_train.isnull().sum() > 0] #a primeira parte pega todas as colunas e soma cada campo e
                                                    # a segunda parte mostra só as colunas q contém 1 ou mais valores

taxi_id                      0
trip_end_timestamp           5
trip_seconds               130
pickup_census_tract       6876
dropoff_census_tract      6988
pickup_community_area     2328
dropoff_community_area    2718
tips                         1
tolls                     3361
extras                       1
trip_total                   1
company                   3548
pickup_latitude           2327
pickup_longitude          2327
pickup_location           2327
dropoff_latitude          2669
dropoff_longitude         2669
dropoff_location          2669
dtype: int64

In [8]:
print(f'dataset total: {taxi_train.shape}')
print(f'dataset sem NA: {taxi_train.dropna().shape}')

dataset total: (30000, 24)
dataset sem NA: (12986, 24)


### Tratamento dos valores ausentes

In [9]:
submission_sem_nulos = submission.copy()

In [10]:
taxi_sem_nulos = taxi_train.copy()

In [11]:
for coluna in submission.columns:
    submission_sem_nulos[coluna] = submission_sem_nulos[coluna].fillna(submission_sem_nulos[coluna].mode()[0])

In [12]:
for coluna in taxi_train.columns:
    taxi_sem_nulos[coluna] = taxi_sem_nulos[coluna].fillna(taxi_sem_nulos[coluna].mode()[0])

In [13]:
print(f"tamanho do dataset sem nulos: {taxi_sem_nulos.shape} \n")
taxi_sem_nulos.isnull().sum()[taxi_sem_nulos.isnull().sum() > 0] 

tamanho do dataset sem nulos: (30000, 24) 



Series([], dtype: int64)

In [14]:
taxi_sem_nulos.describe()

,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
count,30000.000000,30000.000000,3.000000e+04,3.000000e+04,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,792.925100,3.237427,1.703154e+10,1.703153e+10,21.979900,20.050067,12.872645,1.402827,0.000867,0.867840,15.190602,41.898959,-87.656643,41.898463,-87.652311
std,1504.336924,6.297811,3.545074e+05,3.539573e+05,19.498597,17.544597,12.289035,2.640719,0.050319,2.742128,15.146933,0.037517,0.069333,0.037579,0.057329
min,0.000000,0.000000,1.703101e+10,1.703101e+10,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,41.673820,-87.913625,41.660136,-87.913625
25%,300.000000,0.200000,1.703108e+10,1.703108e+10,8.000000,8.000000,6.000000,0.000000,0.000000,0.000000,7.000000,41.880994,-87.651062,41.880994,-87.655879
50%,540.000000,1.190000,1.703184e+10,1.703184e+10,8.000000,8.000000,8.050000,0.000000,0.000000,0.000000,9.650000,41.891972,-87.632746,41.891972,-87.632746
75%,960.000000,3.000000,1.703184e+10,1.703184e+10,32.000000,32.000000,13.500000,2.000000,0.000000,1.000000,15.450000,41.902788,-87.626215,41.907492,-87.626215
max,86386.000000,247.000000,1.703198e+10,1.703198e+10,77.000000,77.000000,269.850000,63.000000,4.000000,99.500000,274.350000,42.009623,-87.534903,42.015934,-87.540936


#### Escolha das variáveis

In [15]:
correlacao = taxi_sem_nulos.corr()

In [16]:
correlacao.style.background_gradient(cmap='coolwarm')

,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
trip_seconds,1.000000,0.304420,0.100857,0.077293,0.170709,0.118341,0.405054,0.197896,0.008344,0.137145,0.388431,0.081174,-0.203902,0.048444,-0.159402
trip_miles,0.304420,1.000000,0.184743,0.149156,0.306037,0.212827,0.637248,0.352218,0.033335,0.281219,0.630469,0.146480,-0.379160,0.086718,-0.297332
pickup_census_tract,0.100857,0.184743,1.000000,0.439650,0.399079,-0.043886,0.272280,0.157670,0.017387,0.121856,0.270772,0.081420,-0.390089,0.055639,-0.103135
dropoff_census_tract,0.077293,0.149156,0.439650,1.000000,-0.005383,0.347706,0.216261,0.099727,0.015044,0.050034,0.201949,0.061796,-0.092235,0.006843,-0.356330
pickup_community_area,0.170709,0.306037,0.399079,-0.005383,1.000000,0.150606,0.366950,0.271848,0.047718,0.290667,0.398657,0.143569,-0.714351,-0.024314,-0.081349
dropoff_community_area,0.118341,0.212827,-0.043886,0.347706,0.150606,1.000000,0.234144,0.115101,-0.011830,0.001161,0.210477,-0.064284,-0.015177,-0.058257,-0.569348
fare,0.405054,0.637248,0.272280,0.216261,0.366950,0.234144,1.000000,0.536122,0.036775,0.357141,0.970554,0.175137,-0.470146,0.084386,-0.349991
tips,0.197896,0.352218,0.157670,0.099727,0.271848,0.115101,0.536122,1.000000,0.030625,0.296683,0.665795,0.103707,-0.330563,0.027283,-0.179216
tolls,0.008344,0.033335,0.017387,0.015044,0.047718,-0.011830,0.036775,0.030625,1.000000,0.109999,0.058359,0.037117,-0.062829,-0.008006,0.005878
extras,0.137145,0.281219,0.121856,0.050034,0.290667,0.001161,0.357141,0.296683,0.109999,1.000000,0.523647,0.140735,-0.370236,-0.015429,-0.053598


#### Modelo de Regressão

In [28]:
x = taxi_sem_nulos[['trip_miles','trip_seconds','pickup_census_tract','dropoff_census_tract','tips','extras', 'trip_total','dropoff_community_area','pickup_community_area','pickup_longitude','dropoff_longitude']]
y = taxi_sem_nulos['fare']

reg = LinearRegression() # instanciando o modelo
reg.fit(x, y) # treinando o modelo


LinearRegression()

In [29]:
print(reg.intercept_, reg.coef_, reg.score(x, y))

-80.64678905620809 [-1.35216382e-03 -9.96252893e-08  2.58903030e-09  9.85972003e-10
 -1.01747085e+00 -1.00315313e+00  1.00077572e+00 -3.56515177e-05
 -4.40406076e-04 -1.38737686e-01 -8.64866986e-02] 0.9998528760466036


In [33]:
resultados = submission_sem_nulos[['trip_miles','trip_seconds','pickup_census_tract','dropoff_census_tract','tips','extras', 'trip_total','dropoff_community_area','pickup_community_area','pickup_longitude','dropoff_longitude']]

In [34]:
resultados['fare_predict'] = reg.predict(resultados)

/tmp/ipykernel_5979/455464575.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultados['fare_predict'] = reg.predict(resultados)


In [35]:
resultados

,trip_miles,trip_seconds,pickup_census_tract,dropoff_census_tract,tips,extras,trip_total,dropoff_community_area,pickup_community_area,pickup_longitude,dropoff_longitude,fare_predict
0,0.9,3180.0,1.703198e+10,1.703108e+10,9.20,5.0,55.20,8.0,76.0,-87.903040,-87.640902,40.847961
1,2.9,540.0,1.703108e+10,1.703107e+10,2.00,0.0,11.44,7.0,8.0,-87.626215,-87.635891,9.381937
2,18.0,540.0,1.703108e+10,1.703108e+10,0.00,1.0,8.65,8.0,8.0,-87.635760,-87.626215,7.601604
3,4.2,1020.0,1.703184e+10,1.703108e+10,2.00,0.0,16.00,8.0,33.0,-87.624135,-87.620763,13.933004
4,2.5,720.0,1.703184e+10,1.703184e+10,0.00,0.0,9.45,33.0,32.0,-87.632746,-87.624135,9.416972
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1.5,240.0,1.703106e+10,1.703106e+10,0.00,1.0,7.05,6.0,6.0,-87.646007,-87.661545,6.028062
19996,26.7,6036.0,1.703184e+10,1.703132e+10,16.20,5.5,97.20,32.0,8.0,-87.632746,-87.620993,75.211240
19997,0.6,240.0,1.703184e+10,1.703108e+10,0.00,0.0,4.65,8.0,32.0,-87.632746,-87.637844,4.617194
19998,0.0,1500.0,1.703184e+10,1.703184e+10,0.00,5.0,30.75,15.0,32.0,-87.625192,-87.763399,25.732669


In [36]:
submission_matheus = pd.DataFrame(columns=['unique_key', 'fare'])

In [37]:
submission_matheus['unique_key'] = submission['unique_key']
submission_matheus['fare'] = resultados['fare_predict']

In [38]:
sample_submission

,unique_key,fare
0,ab1adf76a077d31a82824264aed032f197423f5d,12.0
1,d19b9b5131a749b6c8fee6661a84833421c1e25f,13.0
2,f5a6afb4daedb37b82500baf34cf41adcf7ca167,13.0
3,f06a3db639297271c4a1f5b3704d5c0790d62620,13.0
4,3d9dd92332405f2649b746dda48c2f926477e358,8.0
...,...,...
19995,bd399345026e933c8c9720433f3f7ef70ab2f6d3,16.0
19996,32f21576e43a257fa1c4c624576a097a8ff5f640,5.0
19997,2a66652862821235e5a24e9a6dc28e1663c41fcd,8.0
19998,dbac35b3e6a4ac99f8c691aaf885abc2f43cb4ca,18.0


In [39]:
submission_matheus

,unique_key,fare
0,ab1adf76a077d31a82824264aed032f197423f5d,40.847961
1,d19b9b5131a749b6c8fee6661a84833421c1e25f,9.381937
2,f5a6afb4daedb37b82500baf34cf41adcf7ca167,7.601604
3,f06a3db639297271c4a1f5b3704d5c0790d62620,13.933004
4,3d9dd92332405f2649b746dda48c2f926477e358,9.416972
...,...,...
19995,bd399345026e933c8c9720433f3f7ef70ab2f6d3,6.028062
19996,32f21576e43a257fa1c4c624576a097a8ff5f640,75.211240
19997,2a66652862821235e5a24e9a6dc28e1663c41fcd,4.617194
19998,dbac35b3e6a4ac99f8c691aaf885abc2f43cb4ca,25.732669


In [40]:
submission_matheus.dropna().to_csv('submission_matheus.csv', index=False)